<a href="https://colab.research.google.com/github/umutky/colab_notebooks/blob/main/CrimeDatasetwPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz
# install findspark
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [6]:
import pyspark
from pyspark.sql import SparkSession

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('deneme').getOrCreate()
spark

In [8]:
!wget https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?accessType=DOWNLOAD \
      -O Crime_Data_from_2020_to_Present.csv

--2024-02-12 18:32:32--  https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?accessType=DOWNLOAD
Resolving data.lacity.org (data.lacity.org)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.lacity.org (data.lacity.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘Crime_Data_from_2020_to_Present.csv’

Crime_Data_from_202     [       <=>          ] 217.50M  4.92MB/s    in 46s     

2024-02-12 18:33:18 (4.77 MB/s) - ‘Crime_Data_from_2020_to_Present.csv’ saved [228067544]



In [9]:
data = spark.read.csv('Crime_Data_from_2020_to_Present.csv', header=True, inferSchema=True)

In [10]:
data.printSchema()

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- L

In [11]:
data.show(5, False)

+---------+----------------------+----------------------+--------+----+---------+-----------+--------+------+----------------------------------------+-------------------+--------+--------+------------+---------+--------------------------------------------+--------------+-----------+------+------------+--------+--------+--------+--------+----------------------------------------+------------+-------+---------+
|DR_NO    |Date Rptd             |DATE OCC              |TIME OCC|AREA|AREA NAME|Rpt Dist No|Part 1-2|Crm Cd|Crm Cd Desc                             |Mocodes            |Vict Age|Vict Sex|Vict Descent|Premis Cd|Premis Desc                                 |Weapon Used Cd|Weapon Desc|Status|Status Desc |Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|LOCATION                                |Cross Street|LAT    |LON      |
+---------+----------------------+----------------------+--------+----+---------+-----------+--------+------+----------------------------------------+-------------------+------

**DR_NO**: Unique identifier for each reported crime incident (integer).

**Date Rptd**: Date when the crime was reported (string).

**DATE OCC**: Date when the crime occurred (string).

**TIME OCC**: Time when the crime occurred (integer).

**AREA**: Area code where the crime occurred (integer).

**AREA NAME**: Name of the area where the crime occurred (string).

**Rpt Dist No:** Report district number (integer).

**Part 1-2:** Indicator for Part I or Part II crime (integer).

**Crm Cd:** Crime code (integer).

**Crm Cd Desc:** Description of the crime (string).

**Mocodes**: Modus operandi (method or pattern of operation) for the crime (string).

**Vict Age**: Age of the victim (integer).

**Vict Sex**: Gender of the victim (string).

**Vict Descent**: Descent (ethnicity or nationality) of the victim (string).

**Premis Cd**: Premise code (integer).

**Premis Desc**: Description of the premise where the crime occurred (string).

**Weapon Used Cd**: Code for the weapon used (integer).

**Weapon Desc**: Description of the weapon used (string).

**Status**: Current status of the case (string).

**Status Desc:** Description of the current case status (string).

**Crm Cd 1-4**: Additional crime codes (integer).

**LOCATION**: Location where the crime occurred (string).

**Cross Street**: Cross street information (string).

**LAT**: Latitude coordinate of the crime location (double).

**LON**: Longitude coordinate of the crime location (double).

In [12]:
data.columns

['DR_NO',
 'Date Rptd',
 'DATE OCC',
 'TIME OCC',
 'AREA',
 'AREA NAME',
 'Rpt Dist No',
 'Part 1-2',
 'Crm Cd',
 'Crm Cd Desc',
 'Mocodes',
 'Vict Age',
 'Vict Sex',
 'Vict Descent',
 'Premis Cd',
 'Premis Desc',
 'Weapon Used Cd',
 'Weapon Desc',
 'Status',
 'Status Desc',
 'Crm Cd 1',
 'Crm Cd 2',
 'Crm Cd 3',
 'Crm Cd 4',
 'LOCATION',
 'Cross Street',
 'LAT',
 'LON']

In [13]:
new_column_names = [x.lower().replace(' ', '_') for x in data.columns]
crime_data = data.toDF(*new_column_names)

In [14]:
crime_data.printSchema()

root
 |-- dr_no: integer (nullable = true)
 |-- date_rptd: string (nullable = true)
 |-- date_occ: string (nullable = true)
 |-- time_occ: integer (nullable = true)
 |-- area: integer (nullable = true)
 |-- area_name: string (nullable = true)
 |-- rpt_dist_no: integer (nullable = true)
 |-- part_1-2: integer (nullable = true)
 |-- crm_cd: integer (nullable = true)
 |-- crm_cd_desc: string (nullable = true)
 |-- mocodes: string (nullable = true)
 |-- vict_age: integer (nullable = true)
 |-- vict_sex: string (nullable = true)
 |-- vict_descent: string (nullable = true)
 |-- premis_cd: integer (nullable = true)
 |-- premis_desc: string (nullable = true)
 |-- weapon_used_cd: integer (nullable = true)
 |-- weapon_desc: string (nullable = true)
 |-- status: string (nullable = true)
 |-- status_desc: string (nullable = true)
 |-- crm_cd_1: integer (nullable = true)
 |-- crm_cd_2: integer (nullable = true)
 |-- crm_cd_3: integer (nullable = true)
 |-- crm_cd_4: integer (nullable = true)
 |-- l

In [15]:
crime_data.count()

892934

In [16]:
crime_data.show(5, False)

+---------+----------------------+----------------------+--------+----+---------+-----------+--------+------+----------------------------------------+-------------------+--------+--------+------------+---------+--------------------------------------------+--------------+-----------+------+------------+--------+--------+--------+--------+----------------------------------------+------------+-------+---------+
|dr_no    |date_rptd             |date_occ              |time_occ|area|area_name|rpt_dist_no|part_1-2|crm_cd|crm_cd_desc                             |mocodes            |vict_age|vict_sex|vict_descent|premis_cd|premis_desc                                 |weapon_used_cd|weapon_desc|status|status_desc |crm_cd_1|crm_cd_2|crm_cd_3|crm_cd_4|location                                |cross_street|lat    |lon      |
+---------+----------------------+----------------------+--------+----+---------+-----------+--------+------+----------------------------------------+-------------------+------

In [17]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import to_date
from pyspark.sql.functions import substring
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import from_unixtime

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

crime_data = crime_data.withColumn("date_rptd", to_date("date_rptd", "MM/dd/yyyy"))

crime_data = crime_data.withColumn("date_occ", to_date("date_occ", "MM/dd/yyyy"))

In [18]:
crime_data.printSchema()

root
 |-- dr_no: integer (nullable = true)
 |-- date_rptd: date (nullable = true)
 |-- date_occ: date (nullable = true)
 |-- time_occ: integer (nullable = true)
 |-- area: integer (nullable = true)
 |-- area_name: string (nullable = true)
 |-- rpt_dist_no: integer (nullable = true)
 |-- part_1-2: integer (nullable = true)
 |-- crm_cd: integer (nullable = true)
 |-- crm_cd_desc: string (nullable = true)
 |-- mocodes: string (nullable = true)
 |-- vict_age: integer (nullable = true)
 |-- vict_sex: string (nullable = true)
 |-- vict_descent: string (nullable = true)
 |-- premis_cd: integer (nullable = true)
 |-- premis_desc: string (nullable = true)
 |-- weapon_used_cd: integer (nullable = true)
 |-- weapon_desc: string (nullable = true)
 |-- status: string (nullable = true)
 |-- status_desc: string (nullable = true)
 |-- crm_cd_1: integer (nullable = true)
 |-- crm_cd_2: integer (nullable = true)
 |-- crm_cd_3: integer (nullable = true)
 |-- crm_cd_4: integer (nullable = true)
 |-- locat

In [19]:
print('DataFrame is cached ? : ', crime_data.is_cached)

DataFrame is cached ? :  False


In [20]:
crime_data.cache()
print('DataFrame is cached ? : ', crime_data.is_cached)

DataFrame is cached ? :  True


In [21]:
# unchached
crime_data.unpersist()
print('DataFrame is cached ? : ', crime_data.is_cached)

DataFrame is cached ? :  False
